Link two datasets
=================

Introduction
------------

In [ ]:
!pip install recordlinkage

In [ ]:
import numpy as np
import pandas as pd
import nltk
# nltk.download()
dir(nltk)

['AbstractLazySequence',
 'AffixTagger',
 'AlignedSent',
 'Alignment',
 'AnnotationTask',
 'ApplicationExpression',
 'Assignment',
 'BigramAssocMeasures',
 'BigramCollocationFinder',
 'BigramTagger',
 'BinaryMaxentFeatureEncoding',
 'BlanklineTokenizer',
 'BllipParser',
 'BottomUpChartParser',
 'BottomUpLeftCornerChartParser',
 'BottomUpProbabilisticChartParser',
 'Boxer',
 'BrillTagger',
 'BrillTaggerTrainer',
 'CFG',
 'CRFTagger',
 'CfgReadingCommand',
 'ChartParser',
 'ChunkParserI',
 'ChunkScore',
 'ClassifierBasedPOSTagger',
 'ClassifierBasedTagger',
 'ClassifierI',
 'ConcordanceIndex',
 'ConditionalExponentialClassifier',
 'ConditionalFreqDist',
 'ConditionalProbDist',
 'ConditionalProbDistI',
 'ConfusionMatrix',
 'ContextIndex',
 'ContextTagger',
 'ContingencyMeasures',
 'CoreNLPDependencyParser',
 'CoreNLPParser',
 'Counter',
 'CrossValidationProbDist',
 'DRS',
 'DecisionTreeClassifier',
 'DefaultTagger',
 'DependencyEvaluator',
 'DependencyGrammar',
 'DependencyGraph',
 'Depen

In [ ]:
import recordlinkage
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output



In [ ]:
#Loading the datasets

dfA= pd.read_csv("source_1.csv", sep=",",encoding='UTF-8')
dfB = pd.read_csv("source_2.csv",sep=",",encoding='UTF-8')
dfC = pd.read_csv("matched_data.csv",sep=",",encoding='UTF-8')
validation=pd.read_csv("predicted_matches.csv",sep=",",encoding='UTF-8')

In [ ]:
dfA.head(3)

,id,name
0,0,"Horses, asses, mules and hinnies; live, pure-b..."
1,1,"Horses; live, pure-bred breeding animals"
2,2,"Horses; live, other than pure-bred breeding an..."


In [ ]:
df3 = dfC.rename(columns={"source_1":"feature", "source_2":"labels"})

df3.head()

,feature,labels
0,"Cereals, barley",barley
1,Other fresh or chilled potatoes,potatoes
2,"Fruit, edible; cherries, fresh",cherries
3,"Bran, sharps and other residues, of maize",maize bran
4,"Cereals, millet",millets


In [ ]:
#Lets see unique values in the first dataset
dfA['name'].unique()

array(['Horses, asses, mules and hinnies; live, pure-bred breeding animals',
       'Horses; live, pure-bred breeding animals',
       'Horses; live, other than pure-bred breeding animals', ...,
       'Oth.cathode-ray tubes', 'Other cathode-ray tubes',
       'Tubes for microwaves, magnetrons'], dtype=object)

In [ ]:
#Lets see unique values in the second dataset
dfB['name'].unique()

array(['leveillula lactucae-serriolae', 'podosphaera aphanis',
       'lathyrus czeczottianus', ..., 'fusarium wilt of chickpea',
       'lathyrus tauricola', 'fumaria officinalis subsp. cilicica'],
      dtype=object)

In [ ]:
#Lets see unique values in the matched dataset
df3['labels'].unique()

array(['barley', 'potatoes', 'cherries', 'maize bran', 'millets',
       'sesame seed', 'cigarettes', 'tomatoes', 'beet pulp',
       'sunflower seed', 'figs', 'wheat bran', 'plantains', 'cornflour',
       'processed cheese', 'asparagus', 'avocados', 'lard', 'poppy seed',
       'grape juice', 'equus asinus', 'chestnuts', 'sesame oil',
       'strawberries', 'chickpeas', 'crab apples', 'wheat flour',
       'watermelons', 'sour cherries', 'linseed', 'lentils', 'quinces',
       'garlic', 'cocoa beans', 'beeswax', 'ginger', 'yuca',
       'broken rice', 'coffee substitutes', 'spinach', 'olives',
       'soybeans', 'apricots', 'rye', 'weaning foods', 'tapioca',
       'horses', 'sorghum', 'cottonseed', 'copra', 'sweetpotatoes',
       'eggplants', 'bananas', 'oats', 'mustard seed', 'pears',
       'rabbit meat', 'buckwheat', 'goat meat', 'dates', 'kola nuts',
       'goats', 'buttermilk', 'rice bran', 'pineapples', 'pistachios',
       'raisins', 'cocoa butter', 'coconuts', 'tamari',
  

The datasets are loaded with the following code. The returned datasets
are of type ``pandas.DataFrame``. This makes it easy to manipulate the
data if desired. For details about data manipulation with ``pandas``,
see their comprehensive documentation http://pandas.pydata.org/.

In [ ]:
import nltk
import nltk




print(type(dfA))
print(len(dfA))
print(len(set(dfA)))
print(dfA[:10])
print(dfB[:10])
print(df3[:10])


<class 'pandas.core.frame.DataFrame'>
13238
2
   id                                               name
0   0  Horses, asses, mules and hinnies; live, pure-b...
1   1           Horses; live, pure-bred breeding animals
2   2  Horses; live, other than pure-bred breeding an...
3   4  Horses, asses, mules and hinnies; live, other ...
4   5   Bovine animals; live, pure-bred breeding animals
5   6           Cattle; live, pure-bred breeding animals
6   7  Cattle; live, other than pure-bred breeding an...
7   8          Buffalo; live, pure-bred breeding animals
8   9  Buffalo; live, other than pure-bred breeding a...
9  10  Bovine animals; live, other than pure-bred bre...
   id                            name
0   0   leveillula lactucae-serriolae
1   1             podosphaera aphanis
2   2          lathyrus czeczottianus
3   3  crocus biflorus subsp. caricus
4   4          hordeum brevisubulatum
5   5        vinca major subsp. major
6   6            geranium psilostemon
7   7                 c


Tokenization
-----------------



In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Importing word tokenizer to tokenize our   words 
from nltk.tokenize import word_tokenize 

dfA['clean_names'] = dfA['name'].apply(nltk.word_tokenize)

In [ ]:
from nltk.tokenize import word_tokenize 

dfB['clean_names'] = dfB['name'].apply(nltk.word_tokenize)

In [ ]:
from nltk.tokenize import word_tokenize 

df3['clean_labels'] = df3['labels'].apply(nltk.word_tokenize)

In [ ]:
df3.head()

,feature,labels,clean_labels
0,"Cereals, barley",barley,[barley]
1,Other fresh or chilled potatoes,potatoes,[potatoes]
2,"Fruit, edible; cherries, fresh",cherries,[cherries]
3,"Bran, sharps and other residues, of maize",maize bran,"[maize, bran]"
4,"Cereals, millet",millets,[millets]


In [ ]:
    PAD_token = 0   # Used for padding short sentences
    SOS_token = 1   # Start-of-sentence token
    EOS_token = 2   # End-of-sentence token


Creating a count Corpus doc

In [ ]:
class Vocabulary:
  
    PAD_token = 0   # Used for padding short sentences
    SOS_token = 1   # Start-of-sentence token
    EOS_token = 2   # End-of-sentence token

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            # Word exists; increase word count
            self.word2count[word] += 1
            
    def add_sentence(self, sentence):
        sentence_len = 0
        for word in sentence.split(' '):
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

In [ ]:
voc = Vocabulary('df3')
print(voc)

In [ ]:
corpus = [
'barley','potatoes', 'cherries', 'maize bran', 'millets',
       'sesame seed', 'cigarettes', 'tomatoes', 'beet pulp',
       'sunflower seed', 'figs', 'wheat bran', 'plantains', 'cornflour',
       'processed cheese', 'asparagus', 'avocados', 'lard', 'poppy seed',
       'grape juice', 'equus asinus', 'chestnuts', 'sesame oil',
       'strawberries', 'chickpeas', 'crab apples', 'wheat flour',
       'watermelons', 'sour cherries', 'linseed', 'lentils', 'quinces',
       'garlic', 'cocoa beans', 'beeswax', 'ginger', 'yuca',
       'broken rice', 'coffee substitutes', 'spinach', 'olives',
       'soybeans', 'apricots', 'rye', 'weaning foods', 'tapioca',
       'horses', 'sorghum', 'cottonseed', 'copra', 'sweetpotatoes',
       'eggplants', 'bananas', 'oats', 'mustard seed', 'pears',
       'rabbit meat', 'buckwheat', 'goat meat', 'dates', 'kola nuts',
       'goats', 'buttermilk', 'rice bran', 'pineapples', 'pistachios',
       'raisins', 'cocoa butter', 'coconuts', 'tamari',
       'pawpaw (carica papaya)', 'sheep', 'cranberries', 'cloves',
       'cabbages', 'jute', 'fonio (digitaria)', 'taro', 'raspberries',
       'sugarcane', 'dromedary meat', 'yams', 'safflower seed',
       'palm kernels', 'snails', 'yoghurt', 'quinua', 'yautia', 'carobs',
       'shea nuts', 'plows', 'tur', 'yerba mate', 'maize starch',
       'smoked fish', 'grapefruits', 'butter', 'oleic acid',
       'disc harrows']
print(corpus)


['barley', 'potatoes', 'cherries', 'maize bran', 'millets', 'sesame seed', 'cigarettes', 'tomatoes', 'beet pulp', 'sunflower seed', 'figs', 'wheat bran', 'plantains', 'cornflour', 'processed cheese', 'asparagus', 'avocados', 'lard', 'poppy seed', 'grape juice', 'equus asinus', 'chestnuts', 'sesame oil', 'strawberries', 'chickpeas', 'crab apples', 'wheat flour', 'watermelons', 'sour cherries', 'linseed', 'lentils', 'quinces', 'garlic', 'cocoa beans', 'beeswax', 'ginger', 'yuca', 'broken rice', 'coffee substitutes', 'spinach', 'olives', 'soybeans', 'apricots', 'rye', 'weaning foods', 'tapioca', 'horses', 'sorghum', 'cottonseed', 'copra', 'sweetpotatoes', 'eggplants', 'bananas', 'oats', 'mustard seed', 'pears', 'rabbit meat', 'buckwheat', 'goat meat', 'dates', 'kola nuts', 'goats', 'buttermilk', 'rice bran', 'pineapples', 'pistachios', 'raisins', 'cocoa butter', 'coconuts', 'tamari', 'pawpaw (carica papaya)', 'sheep', 'cranberries', 'cloves', 'cabbages', 'jute', 'fonio (digitaria)', 'taro

In [ ]:
for sent in df3:
  voc.add_sentence(sent)

In [ ]:
import nltk
nltk.download('punkt')

import nltk
nltk.download('averaged_perceptron_tagger')
for i in sent:
     
    # Word tokenizers is used to find the words
    # and punctuation in a string
    wordsList = nltk.word_tokenize(i)
 
 
 
    #  Using a Tagger. Which is part-of-speech
    # tagger or POS-tagger.
    tagged = nltk.pos_tag(wordsList)
 
    print(tagged)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[('c', 'NNS')]
[('l', 'NN')]
[('e', 'NN')]
[('a', 'DT')]
[('n', 'NN')]
[('_', 'NN')]
[('l', 'NN')]
[('a', 'DT')]
[('b', 'NN')]
[('e', 'NN')]
[('l', 'NN')]
[('s', 'NN')]


In [ ]:
for word in range(voc.num_words):
    print(voc.to_word(word))

PAD
SOS
EOS
feature
labels
clean_labels


In [ ]:
for i, word in enumerate(wv.vocab):
    if i == 10:
        break
    print(word)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# list of text documents
text = corpus

vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)

print(vectorizer.vocabulary_)

# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

{'barley': 7, 'potatoes': 78, 'cherries': 20, 'maize': 59, 'bran': 11, 'millets': 62, 'sesame': 90, 'seed': 89, 'cigarettes': 23, 'tomatoes': 108, 'beet': 10, 'pulp': 80, 'sunflower': 103, 'figs': 39, 'wheat': 112, 'plantains': 75, 'cornflour': 29, 'processed': 79, 'cheese': 19, 'asparagus': 4, 'avocados': 5, 'lard': 56, 'poppy': 77, 'grape': 48, 'juice': 52, 'equus': 38, 'asinus': 3, 'chestnuts': 21, 'oil': 66, 'strawberries': 100, 'chickpeas': 22, 'crab': 31, 'apples': 1, 'flour': 41, 'watermelons': 110, 'sour': 96, 'linseed': 58, 'lentils': 57, 'quinces': 81, 'garlic': 44, 'cocoa': 25, 'beans': 8, 'beeswax': 9, 'ginger': 45, 'yuca': 117, 'broken': 12, 'rice': 86, 'coffee': 27, 'substitutes': 101, 'spinach': 98, 'olives': 68, 'soybeans': 97, 'apricots': 2, 'rye': 87, 'weaning': 111, 'foods': 43, 'tapioca': 106, 'horses': 51, 'sorghum': 95, 'cottonseed': 30, 'copra': 28, 'sweetpotatoes': 104, 'eggplants': 37, 'bananas': 6, 'oats': 65, 'mustard': 63, 'pears': 72, 'rabbit': 83, 'meat': 

In [ ]:
coun_vect = CountVectorizer()

count_matrix = coun_vect.fit_transform(text)
count_array = count_matrix.toarray()
test = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names())
print(test)

    acid  apples  apricots  asinus  asparagus  avocados  bananas  barley  \
0      0       0         0       0          0         0        0       1   
1      0       0         0       0          0         0        0       0   
2      0       0         0       0          0         0        0       0   
3      0       0         0       0          0         0        0       0   
4      0       0         0       0          0         0        0       0   
..   ...     ...       ...     ...        ...       ...      ...     ...   
94     0       0         0       0          0         0        0       0   
95     0       0         0       0          0         0        0       0   
96     0       0         0       0          0         0        0       0   
97     1       0         0       0          0         0        0       0   
98     0       0         0       0          0         0        0       0   

    beans  beeswax  ...  tomatoes  tur  watermelons  weaning  wheat  yams  \
0       0 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#Creating Test Data

In [ ]:

train = pd.DataFrame(df3.labels)
test = pd.DataFrame(dfA.name)

In [ ]:
train.head()

,labels
0,barley
1,potatoes
2,cherries
3,maize bran
4,millets


In [ ]:
test.head()

,name
0,"Horses, asses, mules and hinnies; live, pure-b..."
1,"Horses; live, pure-bred breeding animals"
2,"Horses; live, other than pure-bred breeding an..."
3,"Horses, asses, mules and hinnies; live, other ..."
4,"Bovine animals; live, pure-bred breeding animals"


Preprocessing

In [ ]:

# split keywords by separator and save them as array

df3['labels'] = df3['labels'].apply(lambda x: x.split(' '))

# convert description keywords to lowercase
df3['labels'] = df3['labels'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
df3['number_of_labels'] = df3['labels'].apply(lambda row: len(row))

# lets check our keywords
print(df3)

                                              feature           labels  \
0                                     Cereals, barley         [barley]   
1                     Other fresh or chilled potatoes       [potatoes]   
2                      Fruit, edible; cherries, fresh       [cherries]   
3           Bran, sharps and other residues, of maize    [maize, bran]   
4                                     Cereals, millet        [millets]   
..                                                ...              ...   
94  Fish; smoked (whether or not cooked before or ...   [smoked, fish]   
95                             Fresh/dried grapefruit    [grapefruits]   
96  Dairy produce, butter and other fats and oils ...         [butter]   
97  Oleic acid (industrial monocarboxylic fatty ac...    [oleic, acid]   
98  Disc harrows, used agricultural, etc.f/soil pr...  [disc, harrows]   

       clean_labels  number_of_labels  
0          [barley]                 1  
1        [potatoes]            

In [ ]:
# Assign the train split
X_train = train[[train.columns[i] for i in range(train.shape[1]-1) ]]
y_train = train[train.columns[-1]]
# Assign the test split
X_test = test[[test.columns[i] for i in range(train.shape[1]-1) ]]
y_test = test[test.columns[-1]]
# Assign the validation split
X_val = validation[[validation.columns[i] for i in 
                    range(validation.shape[1]-1) ]]
y_val = validation[validation.columns[-1]]

# Print the sets data shapes
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))

X_train shape: (99, 0)
X_test shape: (13238, 0)
y_train shape: (99,)
y_test shape: (13238,)
X_val shape: (15, 1)
y_val shape: (15,)


**Initialize the model**

In [ ]:

import os
import pandas as pd
import nltk
import gensim
from gensim import corpora, models, similarities

x=train['labels'].values.tolist()
y=test['name'].values.tolist()

corpus= x+y
  
tok_corp= [nltk.word_tokenize(text) for text in corpus]
       
           
model = gensim.models.Word2Vec(tok_corp, min_count=1, size = 32)

#model.save('testmodel')
#model = gensim.models.Word2Vec.load('test_model')
#model.most_similar('word')
#model.most_similar([vector])

**Training the Word2Vec Model**

Train the model for reviews. Use a window of size 10 i.e. 10 words before the present word and 10 words ahead. A sentence with at least 2 words should only be considered, configure this using min_count parameter.

Workers define how many CPU threads to be used.

**Build Vocabulary**

In [ ]:
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

(2421795, 3121470)

**Save the Model**

**Finding Similar Words and Similarity between words**

In [ ]:
model.wv.most_similar("fruit")

[('lead', 0.9990960359573364),
 ('leather', 0.999078631401062),
 ('tubes', 0.998991847038269),
 ('metal', 0.9988697171211243),
 ('Other', 0.9987651109695435),
 ('base', 0.9987649321556091),
 ('valves', 0.9987096786499023),
 ('Fresh/chilled', 0.9986755847930908),
 ('machinery', 0.998660147190094),
 ('those', 0.9986047148704529)]

In [ ]:
model.wv.similarity(w1="cereals", w2="animal")

0.99060804

In [ ]:
model.wv.similarity(w1="horse", w2="potatoes")

0.9653631

In [ ]:

#model.save("word2vec.model")

In [ ]:


#Create a dataframe containig the final vectors
vectorized_text = pd.DataFrame(tok_corp)

vectorized_text

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 100 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        
        
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# init word2vec class
from time import sleep
w2v_model = Word2Vec(size = 300,
                     window = 15,
                     min_count = 2,
                     workers = 20,
                     sg = 1,
                     negative = 5,
                     sample = 1e-5)
# build vovab


w2v_model.build_vocab(sent)

import time
  
# train the w2v model
start = time.time()

w2v_model.train(sent, 
                total_examples=w2v_model.corpus_count, 
                epochs=1001, 
                report_delay=1,
                compute_loss = True, # set compute_loss = True
                callbacks=[callback()]) # add the callback class
end = time.time()


print("elapsedtime in seconds :"+ str(end - start))
# save the word2vec model

Loss after epoch 0: 0.0
Loss after epoch 100: 0.0
Loss after epoch 200: 0.0
Loss after epoch 300: 0.0
Loss after epoch 400: 0.0
Loss after epoch 500: 0.0
Loss after epoch 600: 0.0
Loss after epoch 700: 0.0
Loss after epoch 800: 0.0
Loss after epoch 900: 0.0
Loss after epoch 1000: 0.0
elapsedtime in seconds :6.335287094116211


In [ ]:
model.save("word2vec.model")

In [ ]:
reloaded_w2v_model = Word2Vec.load('word2vec.model')
words = list(reloaded_w2v_model.wv.vocab)
print('Vocab size: '+str(len(words)))
w1 = 'animal'
print("Top 3 words similar to animal:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
w1 = 'cereal'
print("Top 3 words similar to cereal:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
print("Similarity between animal and cereal:"+\
      str(reloaded_w2v_model.wv.similarity(w1="animal",w2="cereal")))
print("Similarity between animal and leather:"+\
      str(reloaded_w2v_model.wv.similarity(w1="animal",w2="leather")))

Vocab size: 15654
Top 3 words similar to animal: [('wool', 0.9990599155426025), ('hair', 0.9985626935958862), ('fine', 0.9979420900344849)]
Top 3 words similar to cereal: [('recording', 0.9961655735969543), ('working', 0.9961566925048828), ("'s", 0.996096670627594)]
Similarity between animal and cereal:0.98918587
Similarity between animal and leather:0.98562074


Let us use TSNE to do reduce features

In [ ]:
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    #labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(reloaded_w2v_model)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [ ]:
#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [ ]:
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = vectorized_text.fit_transform(X_train) 
X_test_vectors_tfidf = vectorized_text.transform(X_test)

In [ ]:
df3.head()

#BERT MODEL FOR TEXT CLASSIFICATION FOR UNSPUERVISED DATA

In [ ]:
!pip install bert-serving-server  # server
!pip install bert-serving-client  # client, independent of `bert-serving-server`

In [ ]:

train = pd.DataFrame(df3.labels)
test = pd.DataFrame(dfA.name)

In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
# Assign the train split
X_train = train[[train.columns[i] for i in range(train.shape[1]-1) ]]
y_train = train[train.columns[-1]]
# Assign the test split
X_test = test[[test.columns[i] for i in range(train.shape[1]-1) ]]
y_test = test[test.columns[-1]]
# Assign the validation split
X_val = validation[[validation.columns[i] for i in 
                    range(validation.shape[1]-1) ]]
y_val = validation[validation.columns[-1]]

# Print the sets data shapes
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))



X_train shape: (99, 0)
X_test shape: (13238, 0)
y_train shape: (99,)
y_test shape: (13238,)
X_val shape: (15, 1)
y_val shape: (15,)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
yt = mlb.fit_transform(y)
# Getting a sense of how the tags data looks like
print(yt[0])
print(mlb.inverse_transform(yt[0].reshape(1,-1)))
print(mlb.classes_)

[1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1
 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[(' ', ',', '-', ';', 'H', 'a', 'b', 'd', 'e', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 'u', 'v')]
[' ' '"' '$' '%' '&' "'" '(' ')' '*' '+' ',' '-' '.' '/' '0' '1' '2' '3'
 '4' '5' '6' '7' '8' '9' ':' ';' '<' '=' '>' '?' 'A' 'B' 'C' 'D' 'E' 'F'
 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X'
 'Y' 'Z' '[' ']' '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm'
 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '\x92' '\x93' '\x94'
 '\x96' '\xa0' '°' '³' '´' 'º' 'Á' 'ß' 'á' 'â' 'ã' 'ç' 'è' 'é' 'ê' 'í' 'ô'
 'õ' 'ú']


In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip && unzip uncased_L-12_H-768_A-12.zip

In [ ]:
!bert-serving-start -model_dir uncased_L-12_H-768_A-12/ -num_worker=2 -max_seq_len 50

/usr/local/lib/python3.7/dist-packages/bert_serving/server/helper.py:176: UserWarning: Tensorflow 2.8.0 is not tested! It may or may not work. Feel free to submit an issue at https://github.com/hanxiao/bert-as-service/issues/
  'Feel free to submit an issue at https://github.com/hanxiao/bert-as-service/issues/' % tf.__version__)
usage: /usr/local/bin/bert-serving-start -model_dir uncased_L-12_H-768_A-12/ -num_worker=2 -max_seq_len 50
                 ARG   VALUE
__________________________________________________
           ckpt_name = bert_model.ckpt
         config_name = bert_config.json
                cors = *
                 cpu = False
          device_map = []
       do_lower_case = True
  fixed_embed_length = False
                fp16 = False
 gpu_memory_fraction = 0.5
       graph_tmp_dir = None
    http_max_connect = 10
           http_port = None
        mask_cls_sep = False
      max_batch_size = 256
         max_seq_len = 50
           model_dir = uncased_L-12_H-768_A-12

In [ ]:
from bert_serving.client import BertClient

# make a connection with the BERT server using it's ip address
bc = BertClient()
# get the embedding for train and val sets
X_tr_bert = bc.encode(X_tr.tolist())
X_val_bert = bc.encode(X_val.tolist())

In [ ]:
from sklearn.linear_model import LogisticRegression

# LR model
model_bert = LogisticRegression()
# train
model_bert = model_bert.fit(X_train, y_test)
# predict
pred_bert = model_bert.predict(X_val)

In [ ]:

from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert)

In [ ]:
from bert_serving.client import BertClient

# make a connection with the BERT server using it's ip address; do not give any ip if same computer
bc = BertClient(ip="nEW_sERVER")
# get the embedding
embedding = bc.encode(["I love data science and analytics vidhya."])
# check the shape of embedding, it should be 1x768
print(embedding.shape)

In [ ]:

from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor


 X, y = make_regression(n_samples=10, n_targets=3, random_state=1)
 
 MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X, y).predict(X)


In [ ]:
X = df1.features
y = df3.labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

Training model Classification

In [ ]:

from sklearn.linear_model import LogisticRegression

# LR model
model_bert = LogisticRegression()
# train
model_bert = model_bert.fit(x_tr, y_tr)
# predict
pred_bert = model_bert.predict(X_val_bert)

In [ ]:

from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert))

In [ ]:
# set aside 20% of train and test data for evaluation
X_train, X_test, y_train, y_test = train_test_split(train, test,
    test_size=0.2, shuffle = True, random_state = 8)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 8) # 0.25 x 0.8 = 0.2


print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(y_train.shape))
print("y val shape: {}".format(y_test.shape))

ValueError: ignored

In [ ]:
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)

In [ ]:
import nltk
import nltk




print(type(pairs))
print(len(pairs))
print(len(set(pairs)))


<class 'pandas.core.indexes.multi.MultiIndex'>
647907434


With the method ``index``, all possible (and unique) record pairs are
made. The method returns a ``pandas.MultiIndex``. The number of pairs is
equal to the number of records in ``dfA`` times the number of records in
``dfB``.

In [ ]:
print (len(dfA), len(dfB), len(pairs))

13238 48943 647907434


**Cleaning**

In [ ]:
dfA = pd.DataFrame()
for i,token in enumerate(dfA):
    dfA.loc[i,'name'] = token.name

In [ ]:
for token in dfA:
    print(token)

In [ ]:
## create dtf
df = pd.read_csv("/content/source_1.csv")
## rename columns
df = df.rename(columns={"category":"label", "headline":"text"})
## print 5 random rows
df.sample(5)

,id,name
1187,1482,Stevia rebaudiana
12762,12589,"Other recor.appar./reprod.of sound, of magneti..."
8216,7968,"Oth.rolled prod.iron/steel, w>=6dm, galv.oth.p..."
9942,9730,"Autom.machines f/sorting, optical post code re..."
2949,2322,Tiratricol (triac) and its sodium salt


Many of these record pairs do not belong to the same person. In case of
one-to-one matching, the number of matches should be no more than the
number of records in the smallest dataframe. In case of full indexing,
``min(len(dfA), len(N_dfB))`` is much smaller than ``len(pairs)``. The
``recordlinkage`` module has some more advanced indexing methods to
reduce the number of record pairs. Obvious non-matches are left out of
the index. Note that if a matching record pair is not included in the
index, it can not be matched anymore.

One of the most well known indexing methods is named *blocking*. This
method includes only record pairs that are identical on one or more
stored attributes of the person (or entity in general). The blocking
method can be used in the ``recordlinkage`` module.

In [ ]:
#Add a block index.

from recordlinkage.index import Block

indexer = recordlinkage.Index()
indexer.add(Block())

In [ ]:
!pip install BlockIndex

from recordlinkage.index import Block

indexer = recordlinkage.BlockIndex(on='name')
indexer.index(dfA, dfB)

ERROR: Could not find a version that satisfies the requirement BlockIndex (from versions: none)
ERROR: No matching distribution found for BlockIndex


AttributeError: ignored

In [ ]:
#Add a Sorted Neighbourhood Index.

from recordlinkage.index import SortedNeighbourhood

indexer = recordlinkage.index.Random
indexer.add(SortedNeighbourhood( left_on='name', right_on='name', window=9))

indexer.index(dfA, dfB)

AttributeError: ignored

The argument "given\_name" is the blocking variable. This variable has
to be the name of a column in ``dfA`` and ``dfB``. It is possible to
parse a list of columns names to block on multiple variables. Blocking
on multiple variables will reduce the number of record pairs even
further.

Another implemented indexing method is *Sorted Neighbourhood Indexing*
(``recordlinkage.index.SortedNeighbourhood``). This method is very
useful when there are many misspellings in the string were used for
indexing. In fact, sorted neighbourhood indexing is a generalisation of
blocking. See the documentation for details about sorted neighbourd
indexing.

Compare records
---------------

Each record pair is a candidate match. To classify the candidate record
pairs into matches and non-matches, compare the records on all
attributes both records have in common. The ``recordlinkage`` module has
a class named ``Compare``. This class is used to compare the records.
The following code shows how to compare attributes.

In [ ]:
compare_cl = recordlinkage.Compare()

compare_cl.string("name", "name", method="jarowinkler", threshold=0.85, label="name")
compare_cl.exact("id", "id", label="id")
features = compare_cl.compute(candidate_links, dfA, dfB)

The comparing of record pairs starts when the ``compute`` method is
called. All attribute comparisons are stored in a DataFrame with
horizontally the features and vertically the record pairs.

In [ ]:
features

,,name,id


In [ ]:
features.describe()

,name,id
count,0.0,0.0
mean,NaN,NaN
std,NaN,NaN
min,NaN,NaN
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,NaN,NaN


The last step is to decide which records belong to the same person. In
this example, we keep it simple:

In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0     1566
5.0     1332
4.0      343
3.0      146
2.0    16427
1.0    57435
dtype: int64

In [ ]:
features[features.sum(axis=1) > 3]

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-2371-org,rec-2371-dup-0,1,1.0,1,1,1,1.0
rec-3024-org,rec-3024-dup-0,1,1.0,1,0,1,0.0
rec-4652-org,rec-4652-dup-0,1,1.0,1,0,1,1.0
rec-4795-org,rec-4795-dup-0,1,1.0,1,1,1,1.0
rec-1016-org,rec-1016-dup-0,1,1.0,1,1,0,1.0
...,...,...,...,...,...,...,...
rec-4528-org,rec-4528-dup-0,1,1.0,1,1,1,1.0
rec-4887-org,rec-4887-dup-0,1,1.0,1,0,1,1.0
rec-4350-org,rec-4350-dup-0,1,1.0,1,1,1,1.0


Full code
---------

In [ ]:
import recordlinkage
from recordlinkage.datasets import load_febrl4

dfA, dfB = load_febrl4()

# Indexation step
indexer = recordlinkage.Index()
indexer.block("given_name")
candidate_links = indexer.index(dfA, dfB)

# Comparison step
compare_cl = recordlinkage.Compare()

compare_cl.exact("given_name", "given_name", label="given_name")
compare_cl.string("surname", "surname", method="jarowinkler", threshold=0.85, label="surname")
compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
compare_cl.exact("suburb", "suburb", label="suburb")
compare_cl.exact("state", "state", label="state")
compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")

features = compare_cl.compute(candidate_links, dfA, dfB)

# Classification step
matches = features[features.sum(axis=1) > 3]
print(len(matches))

3241
